# Imports

In [2]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import datasets
from torchvision.transforms import ToTensor
import torch.optim as optim

import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import pandas as pd
import copy
from tqdm import tqdm
# from tqdm import tqdm_notebook as tqdm
import time
import os
from scipy.spatial import distance
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Bora

In [3]:
path = '/content/drive/MyDrive/Doutorado Unicamp/Projeto/github/tcgan/raw-data/UCR_TS_Archive_2015/Fish/Fish_TRAIN.txt'
data_train = np.genfromtxt(path, delimiter=',', dtype=np.float32)
path = '/content/drive/MyDrive/Doutorado Unicamp/Projeto/github/tcgan/raw-data/UCR_TS_Archive_2015/Fish/Fish_TEST.txt'
data_test = np.genfromtxt(path, delimiter=',', dtype=np.float32)
print(data_train.shape, data_test.shape)

(175, 464) (175, 464)


In [4]:
# parse
x_tr = data_train[:, :-1]
y_tr = data_train[:, -1].astype(int)
x_te = data_test[:, :-1]
y_te = data_test[:, -1].astype(int)
y_all = np.concatenate([y_tr, y_te])
classes, y_all = np.unique(y_all, return_inverse=True)
n_class = len(classes)
x_tr = x_tr[..., np.newaxis]
x_te = x_te[..., np.newaxis]

# all data can be used in unsupervised learning
x_all = np.vstack([x_tr, x_te])
_, x_te_gan, _, _ = train_test_split(
    x_all, y_all, train_size=0.9, random_state=42, stratify=y_all)
x_tr_gan = x_all  # use all

In [5]:
input_shape = x_tr.shape[1:]
input_shape

(463, 1)

In [6]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [7]:
def build_generator_network(generated_sample_shape, input_layer):
    n_layers = 4
    kernel_size = 5
    strides = 2
    g_units_base=32

    steps = generated_sample_shape[0]
    layer_steps = [steps]
    for i in range(n_layers):
        layer_steps.append(int(np.ceil(float(layer_steps[-1]) / float(strides))))
    layer_steps.reverse()

    conv_units = []
    if n_layers > 1:
        conv_units.append(g_units_base)
        for _ in range(n_layers - 2):  # minus the first and the last layers
            conv_units.append(conv_units[-1] * 2)
    conv_units.reverse()
    # the last layer must be aligned to the number of dimensions of input.
    conv_units.append(generated_sample_shape[-1])

    dense_0 = nn.Linear(input_layer, layer_steps[0] * conv_units[0] * 2, device=device)
    dense_0.weight = nn.init.trunc_normal_(dense_0.weight, mean=0.0, std=0.02, a=-0.04, b=0.04)
    batch_0 = nn.BatchNorm1d(layer_steps[0] * conv_units[0] * 2)
    reluu_0 = nn.ReLU()
    resha_0 = nn.Unflatten(1, (layer_steps[0], conv_units[0] * 2))
    assert h.shape[1] == layer_steps[0]


In [35]:
generated_sample_shape = input_shape
noise_shape = 100
input_layer = noise_shape

n_layers = 4
kernel_size = 10
strides = 2
g_units_base=32

steps = generated_sample_shape[0]
layer_steps = [steps]
for i in range(n_layers):
    layer_steps.append(int(np.ceil(float(layer_steps[-1]) / float(strides))))
layer_steps.reverse()

conv_units = []
if n_layers > 1:
    conv_units.append(g_units_base)
    for _ in range(n_layers - 2):  # minus the first and the last layers
        conv_units.append(conv_units[-1] * 2)
conv_units.reverse()
# the last layer must be aligned to the number of dimensions of input.
conv_units.append(generated_sample_shape[-1])

dense_0 = nn.Linear(input_layer, layer_steps[0] * conv_units[0] * 2, device=device)
dense_0.weight = nn.init.trunc_normal_(dense_0.weight, mean=0.0, std=0.02, a=-0.04, b=0.04)


In [9]:
convt = nn.ConvTranspose1d(256, 128, kernel_size, stride=strides, padding=1, output_padding=0, device=device)
convt.weight.shape

torch.Size([256, 128, 10])

In [29]:
convt = nn.ConvTranspose1d(32, 1, kernel_size, stride=2, padding=5, output_padding=1, device=device)
z = torch.randn(15, 32, 232)
convt(z).shape

torch.Size([15, 1, 463])

In [ ]:
# fractional conv layers
conv_units = [256, 128, 64, 32, 1]
for i in range(n_layers):
    name = f'conv_{i}'
    if layer_steps[i] * strides == layer_steps[i + 1]:
        convt = nn.ConvTranspose1d(conv_units[i], conv_units[i+1], kernel_size,
                                   stride=strides, padding=4, device=device)
        convt.weight = nn.init.trunc_normal_(convt.weight, mean=0.0, std=0.02, a=-0.04, b=0.04)
    else:
        convt = nn.ConvTranspose1d(conv_units[i], conv_units[i+1], kernel_size,
                                   stride=strides, padding=5, output_padding=1, device=device)
        convt.weight = nn.init.trunc_normal_(convt.weight, mean=0.0, std=0.02, a=-0.04, b=0.04)
    h = conv(h)
    if i < n_layers - 1:
        # the last layer
        # - does not apply ReLU
        # - does not apply BatchNorm
        h = tf.keras.layers.BatchNormalization()(h)
        h = layers.ReLU(name=f'{name}_relu')(h)
    assert h.shape[1] == layer_steps[i + 1]
assert h.shape[-1] == generated_sample_shape[-1]

In [81]:
conv_units

[128, 64, 32, 1]

In [82]:
model = nn.Sequential(
    nn.Linear(input_layer, layer_steps[0] * conv_units[0] * 2, device=device),
    nn.BatchNorm1d(layer_steps[0] * conv_units[0] * 2, device=device),
    nn.ReLU(),
    nn.Unflatten(1, (conv_units[0] * 2, layer_steps[0])),

    nn.ConvTranspose1d(conv_units[0] * 2, conv_units[0], kernel_size,
                                stride=strides, padding=4, device=device),
    nn.BatchNorm1d(conv_units[0], device=device),
    nn.ReLU(),
    nn.ConvTranspose1d(conv_units[0], conv_units[1], kernel_size,
                                stride=strides, padding=4, device=device),
    nn.BatchNorm1d(conv_units[1], device=device),
    nn.ReLU(),
    nn.ConvTranspose1d(conv_units[1], conv_units[2], kernel_size,
                                stride=strides, padding=4, device=device),
    nn.BatchNorm1d(conv_units[2], device=device),
    nn.ReLU(),
    nn.ConvTranspose1d(conv_units[2], conv_units[3], kernel_size,
                                stride=strides, padding=5, output_padding=1, device=device)

    )
model[0].weight = nn.init.trunc_normal_(model[0].weight, mean=0.0, std=0.02, a=-0.04, b=0.04)
model[4].weight = nn.init.trunc_normal_(model[4].weight, mean=0.0, std=0.02, a=-0.04, b=0.04)
model[7].weight = nn.init.trunc_normal_(model[7].weight, mean=0.0, std=0.02, a=-0.04, b=0.04)
model[10].weight = nn.init.trunc_normal_(model[10].weight, mean=0.0, std=0.02, a=-0.04, b=0.04)
model[13].weight = nn.init.trunc_normal_(model[13].weight, mean=0.0, std=0.02, a=-0.04, b=0.04)

In [83]:
z = torch.randn(15, 100)
aux = model(z)
aux.shape

torch.Size([15, 1, 463])

In [84]:
torch.squeeze(aux).shape

torch.Size([15, 463])

In [85]:
summary(model, (100,))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                 [-1, 7424]         749,824
       BatchNorm1d-2                 [-1, 7424]          14,848
              ReLU-3                 [-1, 7424]               0
         Unflatten-4              [-1, 256, 29]               0
   ConvTranspose1d-5              [-1, 128, 58]         327,808
       BatchNorm1d-6              [-1, 128, 58]             256
              ReLU-7              [-1, 128, 58]               0
   ConvTranspose1d-8              [-1, 64, 116]          81,984
       BatchNorm1d-9              [-1, 64, 116]             128
             ReLU-10              [-1, 64, 116]               0
  ConvTranspose1d-11              [-1, 32, 232]          20,512
      BatchNorm1d-12              [-1, 32, 232]              64
             ReLU-13              [-1, 32, 232]               0
  ConvTranspose1d-14               [-1,

In [42]:
convt = nn.ConvTranspose1d(256, 128, kernel_size, stride=2, padding=4, dilation=1, device=device)
m = nn.Linear(input_layer, layer_steps[0] * conv_units[0] * 2, device=device)
z = torch.randn(15, 100)
aux = model(z)
aux.shape, convt(aux).shape, z.shape, m(z).shape

(torch.Size([15, 256, 29]),
 torch.Size([15, 128, 58]),
 torch.Size([15, 100]),
 torch.Size([15, 7424]))

# Mais

In [ ]:
class ChangEncoder(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv1d(in_channels=3, out_channels=16, kernel_size=3, device=device)
        self.relu1 = nn.LeakyReLU(0.3)
        self.inst1 = nn.InstanceNorm1d(16, device=device)
        self.drop1 = nn.Dropout(p=0.2)

        self.conv2 = nn.Conv1d(in_channels=16, out_channels=16, kernel_size=3, device=device)
        self.relu2 = nn.LeakyReLU(0.3)
        self.inst2 = nn.InstanceNorm1d(16, device=device)
        self.drop2 = nn.Dropout(p=0.2)

        self.conv3 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=5, stride=4, device=device)
        self.relu3 = nn.LeakyReLU(0.3)
        self.inst3 = nn.InstanceNorm1d(32, device=device)
        self.drop3 = nn.Dropout(p=0.2)

        self.conv4 = nn.Conv1d(in_channels=32, out_channels=32, kernel_size=3, stride=1, device=device)
        self.relu4 = nn.LeakyReLU(0.3)
        self.inst4 = nn.InstanceNorm1d(32, device=device)
        self.drop4 = nn.Dropout(p=0.2)

        self.conv5 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=5, stride=4, device=device)
        self.relu5 = nn.LeakyReLU(0.3)
        self.inst5 = nn.InstanceNorm1d(64, device=device)
        self.drop5 = nn.Dropout(p=0.2)

        self.conv6 = nn.Conv1d(in_channels=64, out_channels=100, kernel_size=5, stride=1, device=device)
        self.relu6 = nn.LeakyReLU(0.3)

        self.glap = nn.AvgPool1d(kernel_size=4)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.inst1(x)
        x = self.drop1(x)

        x = self.conv2(x)
        x = self.relu2(x)
        x = self.inst2(x)
        x = self.drop2(x)

        x = self.conv3(x)
        x = self.relu3(x)
        x = self.inst3(x)
        x = self.drop3(x)

        x = self.conv4(x)
        x = self.relu4(x)
        x = self.inst4(x)
        x = self.drop4(x)

        x = self.conv5(x)
        x = self.relu5(x)
        x = self.inst5(x)
        x = self.drop5(x)

        x = self.conv6(x)
        x = self.relu6(x)

        x = self.glap(x)
        x = x.flatten(start_dim=1)

        logits = x
        return logits

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=1, stratify=y)

X_train, X_val, y_train, y_val = train_test_split(
        X_train, y_train, test_size=0.1, random_state=1, stratify=y_train)

y_train = nn.functional.one_hot(torch.tensor(y_train), num_classes=8).to(torch.float32).to(device)
y_val = nn.functional.one_hot(torch.tensor(y_val), num_classes=8).to(torch.float32).to(device)
y_test = nn.functional.one_hot(torch.tensor(y_test), num_classes=8).to(torch.float32).to(device)

X_train = torch.tensor(X_train, dtype=torch.float32, device=device)
X_val = torch.tensor(X_val, dtype=torch.float32, device=device)
X_test = torch.tensor(X_test, dtype=torch.float32, device=device)

In [ ]:
encoder = ChangEncoder().to(device)
classif = ChangClassifier().to(device)

# torch.manual_seed(42)

loss_fn = nn.CrossEntropyLoss()

optimizer = optim.Adam(list(encoder.parameters()) + list(classif.parameters()), lr=0.001)
# optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

n_epochs = 20   # number of epochs to run
batch_size = 125  # size of each batch
batch_start = torch.arange(0, len(X_train), batch_size)

trainLoss = []
trainAcc = []
valLoss = []
valAcc = []

# training loop
for epoch in range(n_epochs):
    encoder.train()
    classif.train()
    with tqdm(batch_start, unit="batch", mininterval=0, disable=False) as bar:
        bar.set_description(f"Epoch {epoch}")
        for start in bar:
            # take a batch
            X_batch = X_train[start:start+batch_size,:,:]
            y_batch = y_train[start:start+batch_size,:]
            # forward pass
            y_pred = classif(encoder(X_batch))
            loss = loss_fn(y_pred, y_batch)
            # backward pass
            optimizer.zero_grad()
            loss.backward()
            # update weights
            optimizer.step()
            # print progress
            bar.set_postfix(loss=float(loss))
    # evaluate accuracy at end of each epoch
    encoder.eval()
    classif.eval()
    y_pred = classif(encoder(X_val))
    valLoss.append(float(loss_fn(y_pred, y_val)))
    labels = torch.nonzero(y_val)[:,1]
    labels_pred = torch.argmax(y_pred, dim=1)
    valAcc.append(multiclass_f1_score(labels_pred, labels, num_classes=8))
    y_pred = classif(encoder(X_train))
    trainLoss.append(float(loss_fn(y_pred, y_train)))
    labels = torch.nonzero(y_train)[:,1]
    labels_pred = torch.argmax(y_pred, dim=1)
    valAcc.append(multiclass_f1_score(labels_pred, labels, num_classes=8))

fig = go.Figure()

fig.add_trace(go.Scatter(y=valLoss, mode="lines", showlegend=True, name='validação'))
fig.add_trace(go.Scatter(y=trainLoss, mode="lines", showlegend=True, name='treino'))

fig.show()